In [5]:
# Make all plots inline.
%matplotlib inline 

# Import dependencies
import folium
from folium.plugins import HeatMap
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from IPython.core.display import display
import folium.colormap as cm
import scipy.stats as st

# Loading datasets

In [3]:
# Load brewery data.
pop_data = pd.read_csv('./wrk_tables/zip_features.csv')
review_data = pd.read_csv('./src_tables/review.csv')

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
beer_data = pd.read_csv('./wrk_tables/beer_eda_all.csv')

In [5]:
# Remove first column
beer_data = beer_data.iloc[:,1:len(beer_data.iloc[1])+1]

In [6]:
beer_data['id'] = beer_data['id'].str.replace(',', '')

In [7]:
pop_data.head()

,postal_code_id,year,total,total_m,total_f,under_18_m,x18_to_29_m,x30_to_49_m,x50_to_64_m,x65_plus_m,...,median_household_income,num_white_households,num_family_households,avg_household_size,num_us_citizens,num_english_speakers,num_with_american_ancestry,num_hs_grads,num_bach_degree,num_post_grad
0,147.0,2011,18533.0,8971,9562.0,2476,1616,2248,1551,1080,...,13318.0,482967.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,147.0,2012,18544.0,9040,9504.0,2396,1633,2228,1636,1147,...,13495.0,511004.0,NaN,NaN,NaN,NaN,NaN,4053.0,1595.0,272.0
2,147.0,2013,18450.0,8983,9467.0,2319,1576,2260,1651,1177,...,12041.0,549100.0,NaN,NaN,NaN,NaN,NaN,3957.0,1630.0,271.0
3,147.0,2014,18088.0,8824,9264.0,2215,1518,2204,1681,1206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3693.0,1617.0,361.0
4,147.0,2015,17982.0,8788,9194.0,2142,1531,2138,1722,1255,...,10833.0,561048.0,NaN,NaN,NaN,NaN,NaN,3788.0,1720.0,372.0


In [8]:
review_data.head()

,id,beer_id,date,rating,text,user_name,location_id,country
0,"47,879",452,2010-03-26,3.9,Pours brown/black with a thin brown/beige head...,Gustav1996,"11,242",USA
1,"47,880",719,2015-06-21,4.2,On tap. Pours clear red with tan head. Aroma i...,Vestlake,"11,242",USA
2,"47,881",288,2008-06-03,4.2,Bottle. A dark orange beer with no head. The a...,CaptBier,"11,242",USA
3,"47,882","4,357",2013-12-22,3.6,Bottle. Poured amber-brown with a thinnish tan...,coolpop,"11,242",USA
4,"47,883","1,804",2010-02-25,3.2,"UPDATED: MAR 14, 2010 33cl bottle. Poured a oi...",Gustav1996,"11,242",USA


In [10]:
# Replacing missing IBU values with mean value for corresponding beer style
for i in range(0, len(beer_data['id'])):
    if np.isnan(beer_data['ibu'].iloc[i]):
        beer_data['ibu'].iloc[i] = np.nanmean(beer_data['ibu'].loc[beer_data['beer_style_norm'] == beer_data['beer_style_norm'].iloc[i]])

//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
//anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


In [11]:
# None of the Sakes have an IBU, so replace with 0
beer_data['ibu'] = beer_data['ibu'].fillna(0)

In [12]:
# Replacing missing calorie values with mean value for corresponding beer style
for i in range(0, len(beer_data['id'])):
    if np.isnan(beer_data['calories'].iloc[i]):
        beer_data['calories'].iloc[i] = np.nanmean(beer_data['calories'].loc[beer_data['beer_style_norm'] == beer_data['beer_style_norm'].iloc[i]])

In [13]:
# Replacing missing ABV values with mean value for corresponding beer style
for i in range(0, len(beer_data['id'])):
    if np.isnan(beer_data['abv'].iloc[i]):
        beer_data['abv'].iloc[i] = np.nanmean(beer_data['abv'].loc[beer_data['beer_style_norm'] == beer_data['beer_style_norm'].iloc[i]])

In [14]:
beer_data.to_csv('./src_tables/beer_data_all_v2.csv')

# City Geo Data 

In [11]:
from urllib2 import Request, urlopen
import json
from pandas.io.json import json_normalize

In [12]:
with open('./src_tables/cities.json') as data_file:    
    city_coord = json.load(data_file)

city_coord = json_normalize(city_coord)
city_coord.to_csv('./src_tables/city_coord.csv')

In [13]:
city_pop_16 = pd.read_csv('./src_tables/city_pop_2016.csv')

In [14]:
city_merge = pd.merge(city_coord, city_pop_16, left_on = ('city','state'), right_on = ('City','State'), left_index = False)

In [15]:
city_merge.head(25)

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state,Rank,City,State,Census_2010,Base_2010,July_2010,July_2011,July_2012,July_2013,July_2014,July_2015,July_2016
0,New York,4.8%,40.712784,-74.005941,8405837,1,New York,1,New York,New York,8175133,8174962,8192026,8284098,8361179,8422460,8471990,8516502,8537673
1,Los Angeles,4.8%,34.052234,-118.243685,3884307,2,California,2,Los Angeles,California,3792621,3792584,3796292,3825393,3858137,3890436,3920173,3949149,3976322
2,Chicago,-6.1%,41.878114,-87.629798,2718782,3,Illinois,3,Chicago,Illinois,2695598,2695620,2697736,2705404,2714120,2718887,2718530,2713596,2704958
3,Houston,11.0%,29.760427,-95.369803,2195914,4,Texas,4,Houston,Texas,2099451,2100277,2105625,2132157,2166458,2204406,2243999,2284816,2303482
4,Philadelphia,2.6%,39.952584,-75.165222,1553165,5,Pennsylvania,6,Philadelphia,Pennsylvania,1526006,1526006,1528427,1539022,1550379,1555868,1560609,1564964,1567872
5,Phoenix,14.0%,33.448377,-112.074037,1513367,6,Arizona,5,Phoenix,Arizona,1445632,1447624,1450629,1469353,1499007,1525562,1554179,1582904,1615017
6,San Antonio,21.0%,29.424122,-98.493628,1409019,7,Texas,7,San Antonio,Texas,1327407,1327538,1333952,1359002,1385250,1411652,1439150,1468037,1492510
7,San Diego,10.5%,32.715738,-117.161084,1355896,8,California,8,San Diego,California,1307402,1301722,1306153,1320686,1338983,1358242,1379299,1390915,1406630
8,Dallas,5.6%,32.776664,-96.796988,1257676,9,Texas,9,Dallas,Texas,1197816,1197824,1200711,1218664,1241624,1258016,1277376,1297327,1317929
9,San Jose,10.5%,37.338208,-121.886329,998537,10,California,10,San Jose,California,945942,952555,955290,970823,984151,1001580,1013237,1022627,1025350


In [356]:
# removing unwanted columns
city_data = city_merge[['Rank','city','state','latitude','longitude','July_2016']]

In [357]:
# Adding column for number of breweries
city_data['breweries_5'] = [0] * len(city_data)
city_data['breweries_10'] = [0] * len(city_data)
city_data['breweries_25'] = [0] * len(city_data)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

In [358]:
city_data = city_data.sort_values(by = 'Rank', ascending=True)

In [359]:
# Adding column for number of breweries
city_data['ratings_5'] = [0] * len(city_data)
city_data['ratings_10'] = [0] * len(city_data)
city_data['ratings_25'] = [0] * len(city_data)

city_data['review_count_5'] = [0] * len(city_data)
city_data['review_count_10'] = [0] * len(city_data)
city_data['review_count_25'] = [0] * len(city_data)

city_data['price_5'] = [0] * len(city_data)
city_data['price_10'] = [0] * len(city_data)
city_data['price_25'] = [0] * len(city_data)

# Working with Yelp Data

In [319]:
yelp_data = pd.read_csv('./wrk_tables/yelp_brewery.csv')

In [320]:
# Function calculating the Bayesian weighted average yelp rating
# S = wR + (1-w)C
# w = /(v+m), where v = num ratings for each individual beer, and 
# m = avg num ratings for city
# R = overall_rating for each individual beer
# C = average overall_rating for the city
def wtd_yelp_rating():
    rating_list = [0] * len(yelp_data)
    for i in range(0,len(yelp_data)):
        v = yelp_data['review_count'].iloc[i]
        m = np.mean(yelp_data['review_count'].loc[(yelp_data['city'] == yelp_data['city'].iloc[i]) & (yelp_data['state'] == yelp_data['state'].iloc[i])])
        w = v/(v+m)
        R = yelp_data['rating'].iloc[i]
        C = np.mean(yelp_data['rating'].loc[(yelp_data['city'] == yelp_data['city'].iloc[i]) & (yelp_data['state'] == yelp_data['state'].iloc[i])])
        rating_list[i] = w*R + (1-w)*C
    return rating_list

In [321]:
yelp_data['wtd_rating'] = wtd_yelp_rating()

In [322]:
# Checking for duplicates, and there are 451
pd.concat(g for _, g in yelp_data.groupby(["name_norm","city","state"]) if len(g) > 1)

,uid,name,price,rating,review_count,city,state,zip_code,latitude,longitude,name_norm,id,brewery_id,wtd_rating
2956,aces-and-ales-las-vegas,Aces & Ales,$$,4.0,242,Las Vegas,NV,89121,36.121440,-115.064550,Aces & Ales,"7,281",NaN,3.974869
6909,aces-and-ales-las-vegas-2,Aces & Ales,$$,4.0,348,Las Vegas,NV,89128,36.211123,-115.249689,Aces & Ales,"6,992",NaN,3.979113
0,airways-brewing-kent,Airways Brewing,$,4.5,53,Kent,WA,98031,47.411718,-122.224823,Airways Brewing Company,"4,869",104,4.194444
1,airways-brewing-the-bistro-in-downtown-kent-kent,Airways Brewing: The Bistro in Downtown Kent,$$,4.0,182,Kent,WA,98032,47.382172,-122.235910,Airways Brewing Company,"4,862",104,3.976793
6,alaskan-brewing-company-juneau,Alaskan Brewing Company,$,4.5,83,Juneau,AK,99801,58.357030,-134.488990,Alaskan Brewing Company,"4,914",120,4.500000
7,alaskan-brewing-company-depot-juneau,Alaskan Brewing Company Depot,$$,4.5,4,Juneau,AK,99801,58.299744,-134.404330,Alaskan Brewing Company,"4,916",120,4.500000
31,alpine-beer-company-pub-alpine-6,Alpine Beer Company - Pub,$$,4.0,121,Alpine,CA,91901,32.837049,-116.776349,Alpine Beer Company (CA),"2,504",202,3.688669
32,alpine-beer-company-alpine,Alpine Beer Company,$$,4.0,444,Alpine,CA,91901,32.835472,-116.765140,Alpine Beer Company (CA),"2,503",202,3.844899
33,alpine-brewery-alpine,Alpine Brewery,$$,2.5,34,Alpine,CA,91901,32.836880,-116.776627,Alpine Beer Company (CA),"2,505",202,3.354494
39,alulu-brewpub-chicago,Alulu Brewpub,$$,4.0,21,Chicago,IL,60608,41.854728,-87.663522,Alulu Brew,"11,320",226,4.316606


In [323]:
# Converting price '$' to interval data
yelp_data['price'] = yelp_data['price'].map({'$$$$': 4, '$$$': 3, '$$': 2, '$': 1})

In [324]:
# Replacing missing Price values with mean value for city/state
for i in range(0, len(yelp_data)):
    if np.isnan(yelp_data['price'].iloc[i]):
        yelp_data['price'].iloc[i] = np.nanmean(yelp_data['price'].loc[(yelp_data['city'] == yelp_data['city'].iloc[i])
                                                                      & (yelp_data['state'] == yelp_data['state'].iloc[i])])

//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
//anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


In [325]:
# 197 breweries in cities without price information
np.sum(np.isnan(yelp_data['price']))

197

In [326]:
# Since they are likely in small cities, replace remaining NaNs with 1
yelp_data['price'] = yelp_data['price'].fillna(1)

In [165]:
# columns for count of beers by brewery for each style
"""yelp_data['other'] = [0]*len(yelp_data)
yelp_data['sour'] = [0]*len(yelp_data)
yelp_data['wheat'] = [0]*len(yelp_data)
yelp_data['saison'] = [0]*len(yelp_data)
yelp_data['pale'] = [0]*len(yelp_data)
yelp_data['ipa'] = [0]*len(yelp_data)
yelp_data['strong'] = [0]*len(yelp_data)
yelp_data['brown'] = [0]*len(yelp_data)
yelp_data['porter'] = [0]*len(yelp_data)
yelp_data['stout'] = [0]*len(yelp_data)
yelp_data['bock'] = [0]*len(yelp_data)
yelp_data['lager'] = [0]*len(yelp_data)
yelp_data['sake'] = [0]*len(yelp_data)"""

# columns for count of beers by brewery for each ABV strength
yelp_data

,uid,name,price,rating,review_count,city,state,zip_code,latitude,longitude,name_norm,id,brewery_id,wtd_rating
0,airways-brewing-kent,Airways Brewing,1.0,4.5,53,Kent,WA,98031,47.411718,-122.224823,Airways Brewing Company,"4,869",104,4.194444


In [327]:
yelp_data.to_csv('./wrk_tables/yelp_data.csv')

In [182]:
# consider filtering for yelp breweries with at least 10 or 15 reviews
len(yelp_data.loc[yelp_data['review_count'] >= 15])

7565

In [242]:
city_rating = yelp_data.groupby(['city', 'state'])[['wtd_rating']].mean()

# Calculating Distances

In [367]:
# defining distance calculation
def brewery_count(population_limit):
    city_subset = city_data.loc[city_data['July_2016'] >= population_limit]
    R = 3961 # radius of earth in miles
    for i in range(0,len(city_subset)):
        for j in range(0,len(yelp_data)):
            lat1 = np.radians(city_subset['latitude'].iloc[i])
            lat2 = np.radians(yelp_data['latitude'].iloc[j])
            lon1 = np.radians(city_subset['longitude'].iloc[i])
            lon2 = np.radians(yelp_data['longitude'].iloc[j])
            dlon = lon2 - lon1 
            dlat = lat2 - lat1 
            a = (np.sin(dlat/2))**2 + np.cos(lat1) * np.cos(lat2) * (np.sin(dlon/2))**2 
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)) 
            d = R * c
            if d <= 5:
                city_subset['breweries_5'].iloc[i] += 1
                city_subset['breweries_10'].iloc[i] += 1
                city_subset['breweries_25'].iloc[i] += 1
                
                city_subset['ratings_5'].iloc[i] += yelp_data['wtd_rating'].iloc[j]
                city_subset['ratings_10'].iloc[i] += yelp_data['wtd_rating'].iloc[j]
                city_subset['ratings_25'].iloc[i] += yelp_data['wtd_rating'].iloc[j]

                city_subset['review_count_5'].iloc[i] += yelp_data['review_count'].iloc[j]
                city_subset['review_count_10'].iloc[i] += yelp_data['review_count'].iloc[j]
                city_subset['review_count_25'].iloc[i] += yelp_data['review_count'].iloc[j]

                city_subset['price_5'].iloc[i] += yelp_data['price'].iloc[j]
                city_subset['price_10'].iloc[i] += yelp_data['price'].iloc[j]
                city_subset['price_25'].iloc[i] += yelp_data['price'].iloc[j]
            elif d <= 10:
                city_subset['breweries_10'].iloc[i] += 1
                city_subset['breweries_25'].iloc[i] += 1
                
                city_subset['ratings_10'].iloc[i] += yelp_data['wtd_rating'].iloc[j]
                city_subset['ratings_25'].iloc[i] += yelp_data['wtd_rating'].iloc[j]
                
                city_subset['review_count_10'].iloc[i] += yelp_data['review_count'].iloc[j]
                city_subset['review_count_25'].iloc[i] += yelp_data['review_count'].iloc[j]
                
                city_subset['price_10'].iloc[i] += yelp_data['price'].iloc[j]
                city_subset['price_25'].iloc[i] += yelp_data['price'].iloc[j]
            elif d <= 25:
                city_subset['breweries_25'].iloc[i] += 1
                
                city_subset['ratings_25'].iloc[i] += yelp_data['wtd_rating'].iloc[j]
                
                city_subset['review_count_25'].iloc[i] += yelp_data['review_count'].iloc[j]
                    
                city_subset['price_25'].iloc[i] += yelp_data['price'].iloc[j]
            else:
                pass
    city_subset['ratings_5'] =  city_subset['ratings_5']/city_subset['breweries_5']
    city_subset['ratings_10'] =  city_subset['ratings_10']/city_subset['breweries_10']
    city_subset['ratings_25'] =  city_subset['ratings_25']/city_subset['breweries_25']

    city_subset['review_count_5'] = city_subset['review_count_5']/city_subset['breweries_5']
    city_subset['review_count_10'] = city_subset['review_count_10']/city_subset['breweries_10']
    city_subset['review_count_25'] = city_subset['review_count_25']/city_subset['breweries_25']

    city_subset['price_5'] = city_subset['price_5']/city_subset['breweries_5']
    city_subset['price_10'] = city_subset['price_10']/city_subset['breweries_10']
    city_subset['price_25'] = city_subset['price_25']/city_subset['breweries_25']
    return(city_subset)

In [368]:
brew_city_data = brewery_count(100000)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

In [369]:
brew_city_data = brew_city_data.sort_values(by = 'Rank', ascending=True)

In [379]:
brew_city_data['brew_per_100k_5'] = brew_city_data['breweries_5']/brew_city_data['July_2016']*100000
brew_city_data['brew_per_100k_10'] = brew_city_data['breweries_10']/brew_city_data['July_2016']*100000
brew_city_data['brew_per_100k_25'] = brew_city_data['breweries_25']/brew_city_data['July_2016']*100000

brew_city_data['review_per_100k_5'] = brew_city_data['review_count_5']/brew_city_data['July_2016']*100000
brew_city_data['review_per_100k_10'] = brew_city_data['review_count_10']/brew_city_data['July_2016']*100000
brew_city_data['review_per_100k_25'] = brew_city_data['review_count_25']/brew_city_data['July_2016']*100000

In [380]:
brew_city_data.to_csv('./wrk_tables/brew_city_data.csv')

# Storing User Beer

In [44]:
beer_data.loc[beer_data['name'] == 'Dogfish Head 90 Minute Imperial IPA'].index

Int64Index([1165], dtype='int64')

In [45]:
# User searches for beer in database (with autocomplete)
# Here use 'Dogfish Head 90 Minute Imperial IPA' as example (beer_id = 1)
USER_BEER = beer_data.iloc[1165]
USER_BEER

id                                                                1195
brewery_id                                                       4,327
beer_style_id                                                       30
brewery_type_id                                                      4
post_code                                                       24,767
loc_id                                                           17899
name                               Dogfish Head 90 Minute Imperial IPA
description          Esquire Magazine calls our 90 Minute I.I.PA., ...
abv                                                                  9
calories                                                           270
ibu                                                                 90
num_ratings                                                       4646
overall_rating                                                     100
beer_style_rating                                                   99
mean_r

In [46]:
USER_BEER_ID = USER_BEER['id']

# Calculating Style Similarity Score

In [47]:
# Calculate standard deviations of ABV, Calories, IBU, Style Score and Style Rating
abv_std = np.std(beer_data['abv'])
cal_std = np.std(beer_data['calories'])
ibu_std = np.std(beer_data['ibu'])
style_std = np.std(beer_data['style_score'])
style_rat_std = np.std(beer_data['beer_style_rating'])

In [48]:
# Determine relative weights
# I think similarity of style and ABV should be most important
# Then style rating, calories and IBUs (because many IBUs are just the mean)

style_wt = 0.25
abv_wt = 0.3
cal_wt = 0.15
ibu_wt = 0.15
style_rat_wt = 0.15

In [49]:
# Function running the algorithm for getting style similarity scores
# Runs quickly, so no need to pre-compute I think
def style_similarity(user_beer):
    sim_list = [0] * len(beer_data['id'])
    for i in range(0,len(beer_data['id'])):
        if user_beer['id'] == beer_data['id'].iloc[i]:
            sim_list[i] = np.nan
        else:
            dev = np.sqrt(style_wt*((user_beer['style_score'] - beer_data['style_score'].iloc[i])/style_std)**2 + 
                          abv_wt*((user_beer['abv'] - beer_data['abv'].iloc[i])/abv_std)**2 + 
                          cal_wt*((user_beer['calories'] - beer_data['calories'].iloc[i])/cal_std)**2 + 
                          ibu_wt*((user_beer['ibu'] - beer_data['ibu'].iloc[i])/ibu_std)**2 + 
                          style_rat_wt*((user_beer['beer_style_rating'] - beer_data['beer_style_rating'].iloc[i])/style_rat_std)**2)
            sim_score = 100*((1.25-dev)/1.25) # Aiming to have ~80% of beers above 0, depending on style
            sim_list[i] = sim_score
    return sim_list

In [50]:
# Generate Style similarity scores for the test case
SIMILARITY_SCORES = style_similarity(USER_BEER)

In [51]:
# Using this to return index of max value
import operator
index, value = max(enumerate(SIMILARITY_SCORES), key=operator.itemgetter(1))
print(index, value)

(3661, 94.442906407451559)


In [52]:
# Best similarity match
SIMILARITY_SCORES[1783]

18.993169509063446

In [53]:
beer_data.iloc[1783]

id                                                                1834
brewery_id                                                       5,142
beer_style_id                                                       53
brewery_type_id                                                      6
post_code                                                       13,093
loc_id                                                           22816
name                   Flossmoor Station Killer Kowalski Baltic Porter
description          Our deep, dark, full-bodied lager. A slightly ...
abv                                                                  8
calories                                                           240
ibu                                                            48.8548
num_ratings                                                        220
overall_rating                                                      98
beer_style_rating                                                   94
mean_r

# Rating Component

In [54]:
# Function calculating the Bayesian weighted average overall rating
# S = wR + (1-w)C
# w = /(v+m), where v = num ratings for each individual beer, and 
# m = avg num ratings for beer style
# R = overall_rating for each individual beer
# C = average overall_rating for the beer style
def weighted_rating(rtg_measure):
    rating_list = [0] * len(beer_data['id'])
    for i in range(0,len(beer_data['id'])):
        v = beer_data['num_ratings'].iloc[i]
        m = np.mean(beer_data['num_ratings'].loc[beer_data['beer_style_norm'] == beer_data['beer_style_norm'].iloc[i]])
        w = v/(v+m)
        R = beer_data[rtg_measure].iloc[i]
        C = np.mean(beer_data[rtg_measure].loc[beer_data['beer_style_norm'] == beer_data['beer_style_norm'].iloc[i]])
        rating_list[i] = w*R + (1-w)*C
    return rating_list

In [55]:
# Compile ratings based on overall_rating
RATING_SCORES = weighted_rating('overall_rating')

In [56]:
RATING_SCORES[1783]

98.337009007942669

In [57]:
RATING_SCORES[1165]

99.909981663540222

# Calculate Preference Match Component

In [60]:
# Rohan's preferencing matching table loaded here
# containing top 50 matches for each beer_id
# all others would receive a score of 0

PREF_TABLE = pd.read_csv('./wrk_tables/beer_similarities_complete.csv')


In [61]:
# Function to calculate similarity scores for user input beer
def review_match(user_beer):
    pref_list = [0] * len(beer_data['id'])
    pref_list[0] = 0
    pref_scores = PREF_TABLE.loc[PREF_TABLE['beer_id'] == int(user_beer['id'])]
    for i in range(1,len(beer_data)):
        if beer_data['id'].iloc[i] == user_beer['id']:
            pref_list[i] = np.nan
        elif int(beer_data['id'].iloc[i]) not in pref_scores['similar_beer_id'].values:
            pref_list[i] = 0
        else:
            x = pref_scores['similarity_score'].loc[pref_scores['similar_beer_id'] == int(beer_data['id'].iloc[i])].values
            pref_list[i] = x[0]
    pref_list = [i * 100 for i in pref_list]
    return pref_list

In [62]:
PREF_SCORES = review_match(USER_BEER)

In [76]:
len([i for i in PREF_SCORES if i == 0])

854

In [990]:
# Find the most comparable beer based on reviews
index, value = max(enumerate(PREF_SCORES), key=operator.itemgetter(1))
print(index, value)

(1178, 97.152757600000001)


In [991]:
# Check if this makes sense
beer_data.iloc[1178]
# Based on a quick glance over RateBeer descriptions, I think this is a pretty good match
# Despite completely different styles (Imperial IPA vs. Doppelbock), there are similarities

id                                                                1210
brewery_id                                                         667
beer_style_id                                                       66
brewery_type_id                                                      4
post_code                                                       34,355
loc_id                                                            6960
name                 Ballast Point Navigator Doppelbock - Brandy Ba...
description          Navigator Doppelbock is our interpretation of ...
abv                                                                 10
calories                                                           300
ibu                                                            34.6897
num_ratings                                                         43
overall_rating                                                      99
beer_style_rating                                                  100
mean_r

# Calculate Final Ranking Score (out of 100)

In [992]:
# Create a new dataframe specifically for this test (Dogfish Head 90 min)
beer_data_dfh90 = beer_data

In [995]:
# Appending the rating scores to the existing dataframe
beer_data_dfh90['review_sim'] = PREF_SCORES
beer_data_dfh90['style_sim'] = SIMILARITY_SCORES
beer_data_dfh90['weighted_rating'] = RATING_SCORES

In [996]:
# Placeholders for User-input weights between review similarity, style similarity, and overall rating
REVIEW_WEIGHT = 0.40
STYLE_WEIGHT = 0.40
RATING_WEIGHT = 0.20

In [997]:
# Applying the weights to each of the score categories
beer_data_dfh90['review_sim'] = [i * REVIEW_WEIGHT for i in beer_data_dfh90['review_sim']] 
beer_data_dfh90['style_sim'] = [i * STYLE_WEIGHT for i in beer_data_dfh90['style_sim']]
beer_data_dfh90['weighted_rating'] = [i * RATING_WEIGHT for i in beer_data_dfh90['weighted_rating']] 

In [998]:
# Calculate fina score for ranking beers
beer_data_dfh90['final_score'] = beer_data_dfh90['style_sim'] + beer_data_dfh90['review_sim'] + beer_data_dfh90['weighted_rating']

In [999]:
# Create new dataframe to sort the top scores and display
ranking_dfh90 = beer_data_dfh90.sort_values(by = 'final_score', ascending=False)
ranking_dfh90[['id','name','beer_style_norm','abv','ibu','calories','overall_rating','beer_style_rating','review_sim','style_sim','weighted_rating','final_score']].head(25)

,id,name,beer_style_norm,abv,ibu,calories,overall_rating,beer_style_rating,review_sim,style_sim,weighted_rating,final_score
1783,1834,Flossmoor Station Killer Kowalski Baltic Porter,Porter,8.00,48.854839,240.0,98,94,36.845834,7.597268,19.667402,64.110504
2223,2283,Southampton Abbot 12,Strong Ale,10.00,36.430233,300.0,99,95,38.673899,1.302846,19.559658,59.536404
2463,2569,OEC Aeris,Brown/Amber Ale,6.20,57.000000,186.0,97,82,36.157861,5.256414,17.647247,59.061522
3661,3963,Stone Sublimely Self Righteous Black IPA,IPA,8.70,90.000000,261.0,100,100,0.000000,37.777163,19.965549,57.742711
2231,2294,Struise XXX Rye Quad Reserva (Bourbon Barrel A...,Strong Ale,10.00,36.430233,300.0,98,94,36.880970,1.203374,19.457949,57.542293
2258,2317,Le Trou du Diable Buteuse Brassin Sp??cial (Br...,Strong Ale,10.00,36.430233,300.0,99,100,36.331456,1.469205,19.482734,57.283394
2252,2312,Mikkeller Nelson Sauvignon,Strong Ale,9.00,36.430233,270.0,100,100,35.471244,2.062699,19.667086,57.201029
2211,2288,Chimay Grande R??serve Vieillie En Barriques (...,Strong Ale,10.50,36.430233,315.0,99,95,36.739521,0.576670,19.368792,56.684983
2219,2279,Lost Abbey Judgment Day,Strong Ale,10.50,36.430233,315.0,99,96,36.145332,0.652784,19.650972,56.449088
3692,3990,Nickel Brook Malevolent Black Imperial India P...,IPA,9.50,90.000000,285.0,99,99,0.000000,36.631640,19.760023,56.391663


In [1000]:
ranking_dfh90.to_csv('./rank_tables/ranking_dfh90.csv')

# Zip Code Directory

In [447]:
# load zip_code data and geo coordinates
zip_data = pop_data.loc[:, ['city','state','zip_code','lat','long']]
zip_data = zip_data.drop_duplicates(['zip_code'], keep = 'first')

In [448]:
# Convert zip code to string
zip_data['zip_code'] = zip_data.zip_code.apply(str)

# add zeros where necessary to have 5 digits
zip_data['zip_code'] = [str('00'+i) if len(i) == 3 else i for i in zip_data['zip_code']]
zip_data['zip_code'] = [str('0'+i) if len(i) == 4 else i for i in zip_data['zip_code']]

In [576]:
# At some point, may build in functionality to look up cities 
# within certain distance of a given city/state or zip code
brew_city_data.head(1)

,Rank,city,state,latitude,longitude,July_2016,breweries_5,breweries_10,breweries_25,ratings_5,...,review_count_25,price_5,price_10,price_25,brew_per_100k_5,brew_per_100k_10,brew_per_100k_25,review_per_100k_5,review_per_100k_10,review_per_100k_25
0,1,New York,New York,40.712784,-74.005941,8537673,29,49,112,4.230331,...,141.785714,1.710345,1.685714,1.683929,0.339671,0.573927,1.311833,2.557832,2.125513,1.660707


# Mapping Top Cities and Rankings

In [11]:
brew_city_data = pd.read_csv('./wrk_tables/brew_city_data.csv')

In [12]:
# Formula for calculating std deviations away from mean, and limiting the distance to 4 std deviations
# To be used for calculating the quantity, quality and popularity scores
def stdev_calc(df,j):
    x = [0]*len(df)
    for i in range(0,len(df)):
        if (df.iloc[i,j]-np.nanmean(df.iloc[:,j]))/np.std(df.iloc[:,j]) >= 4.0:
            x[i] = 4.0
        elif (df.iloc[i,j]-np.nanmean(df.iloc[:,j]))/np.std(df.iloc[:,j]) <= -4.0:
            x[i] = -4.0
        else:
            x[i] = (df.iloc[i,j]-np.nanmean(df.iloc[:,j]))/np.std(df.iloc[:,j])
    return x

In [13]:
import branca

# Creating the map object to hold the different layers.

# Starting coordinates to load map view.
start_coordinates = (39.965299, -98.266951)

def beer_map(min_brew, radius, min_pop, quality_wt, quantity_wt, popular_wt):
    MIN_BREWERIES = min_brew
    SEARCH_RADIUS = radius
    MIN_POPULATION = min_pop

    QUALITY_WT = quality_wt
    QUANTITY_WT = quantity_wt
    POPULARITY_WT = popular_wt

# Filtered dataset
    brew_city_data_filtered = brew_city_data.loc[(brew_city_data['breweries_'+str(SEARCH_RADIUS)] >= MIN_BREWERIES) &
                                             (brew_city_data['July_2016'] >= MIN_POPULATION), 
                                             ['city','state','latitude','longitude','July_2016','breweries_'+str(SEARCH_RADIUS),
                                             'brew_per_100k_'+str(SEARCH_RADIUS),'ratings_'+str(SEARCH_RADIUS),'price_'+str(SEARCH_RADIUS),
                                             'review_count_'+str(SEARCH_RADIUS),'review_per_100k_'+str(SEARCH_RADIUS)]]

    brew_city_data_filtered['quant_score'] = [st.norm.cdf((i + j)/2)*100 for i, j in zip(stdev_calc(brew_city_data_filtered,5),stdev_calc(brew_city_data_filtered,6))] 
    brew_city_data_filtered['qual_score'] = [st.norm.cdf((i*3 + j)/4)*100 for i, j in zip(stdev_calc(brew_city_data_filtered,7),stdev_calc(brew_city_data_filtered,8))] 
    brew_city_data_filtered['popular_score'] = [st.norm.cdf((i*3 + j*2)/5)*100 for i, j in zip(stdev_calc(brew_city_data_filtered,9),stdev_calc(brew_city_data_filtered,10))]

    brew_city_data_filtered['total_score'] = (QUANTITY_WT*brew_city_data_filtered['quant_score'] + 
                                          QUALITY_WT*brew_city_data_filtered['qual_score'] + 
                                          POPULARITY_WT*brew_city_data_filtered['popular_score'])

    map_data = brew_city_data_filtered.sort_values(by = 'total_score', ascending=False)
    map_data['ranking'] = [i for i in range(1,len(map_data)+1)]

# Create Map object.
    map = folium.Map(location=start_coordinates,
                 zoom_start=4,
                tiles = 'Mapbox Bright',
                control_scale = 'True')
# Other good tiles: 'Catrodb Positron'

# Create layer group for breweries
    brew_group = folium.FeatureGroup(name='City_Rankings').add_to(map)


# Add circle and popup info for each city
    for row in map_data[0:25].iterrows():
    # Only plot point if lat/long is available.
        if (not np.isnan(row[1]['latitude']) and not np.isnan(row[1]['longitude'])):
            brewery_metadata = """
                <ul>
                    <li><strong>Ranking</strong>: {0}</li>
                    <li><strong>City</strong>: {1}, {2}</li>
                    <li><strong>Population</strong>: {3}</li>
                    <li><strong>Overall Score</strong>: {4}</li>
                    <li><strong>Quality Rating</strong>: {5}</li>
                    <li><strong>Average Yelp Score</strong>: {6}</li>
                    <li><strong>Quantity Rating</strong>: {7}</li>
                    <li><strong>Breweries per 100,000</strong>: {8}</li>
                    <li><strong>Popularity Rating</strong>: {9}</li>
                    <li><strong>Average Yelp Reviews per Brewery</strong>: {10}</li>
                </ul>""".format(
                str(row[1]['ranking']), str(row[1]['city']), str(row[1]['state']), str(row[1]['July_2016']), str(round(row[1]['total_score'],2)),
                str(round(row[1]['qual_score'],2)), str(round(row[1]['ratings_'+str(SEARCH_RADIUS)],2)), str(round(row[1]['quant_score'],2)), 
                str(round(row[1]['brew_per_100k_'+str(SEARCH_RADIUS)],2)), str(round(row[1]['popular_score'],2)), str(round(row[1]['review_count_'+str(SEARCH_RADIUS)],2)))
            iframe = branca.element.IFrame(html=brewery_metadata, width=300, height=200)
            popup = folium.Popup(iframe, max_width=2650)
            folium.CircleMarker(
                location = [row[1]['latitude'], row[1]['longitude']],
                color = 'black', fill_color = 'red', fill_opacity = 1.0/row[1]['ranking'],
                radius = row[1]['total_score']/4,
                popup=popup).add_to(brew_group)
            
    map.save('./brew_count_map.html')
    map_data.to_csv('./rank_tables/map_data.csv')
    return map

In [14]:
# min_brew, radius, min_pop, quality_wt, quantity_wt, popular_wt
beer_map(5,5,350000,0.5,0.25,0.25)

# Brewer Map (Cloropleth)

### Creating GeoDataFrame from GeoJSON Files

In [1]:
import os
import folium
from os import listdir
from os.path import isfile, join
import geopandas
print(folium.__version__)
import branca

STATE_LIST = ["AK","AL","AR","AZ","CA","CO","CT","DC","DE","FL","GA","HI","IA","ID","IL","IN","KS","KY",
              "LA","MA","MD","ME","MI","MN","MO","MS","MT","NC","ND","NE","NH","NJ","NM","NV","NY","OH","OK",
              "OR","PA","RI","SC","SD","TN","TX","UT","VA","VT","WA","WI","WV","WY"]

# Creating merged GeoDataFrame
geo_list = {}
for i in STATE_LIST:
    onlyfiles = [join(str('./Geo JSON/geojson-data/'+i+'/'), f) for f in listdir(str('./Geo JSON/geojson-data/'+i+'/')) if f[-7:] == 'geojson']
    geo_list[i] = onlyfiles


    
GEO_DF = geopandas.GeoDataFrame.from_file(geo_list['AK'][0]).drop([0])

for i in geo_list['AK'][1:]:
    GEO_DF = GEO_DF.append(geopandas.GeoDataFrame.from_file(i).drop([0]))
    
for i in STATE_LIST[1:]:
    for j in geo_list[i]:
        GEO_DF = GEO_DF.append(geopandas.GeoDataFrame.from_file(j).drop([0]))

0.3.0


In [3]:
import json

with open('geo_df_all.txt', 'w') as outfile:
    json.dump(GEO_DF.to_json(), outfile)

In [138]:
import csv

# Load population data with socioeconomic and education info
social_data = pd.read_csv('./wrk_tables/pop_df.csv')

# Reload yelp data
yelp_data = pd.read_csv('./wrk_tables/yelp_brewery.csv')

city_coord = pd.read_csv('./src_tables/city_coord2.csv')

### Defining Function to Generate Map

In [146]:
# Users search for a specific city, which filters the map 
def get_map(city, state, metric, year):
    USER_CITY = city
    USER_STATE_ABV = state
    METRIC = metric
    YEAR = year
    
# Create subset of social data for mapping
    df = social_data.loc[(social_data['city'] == USER_CITY) & 
                     (social_data['state'] == USER_STATE_ABV) & 
                    (social_data['year'] == int(YEAR))]

# Create the list of zip codes
    df['zip_code'] = df['zip_code'].apply(str)
    df['zip_code'] = [str('00'+i) if len(i) == 3 else i for i in df['zip_code']]
    df['zip_code'] = [str('0'+i) if len(i) == 4 else i for i in df['zip_code']]

    zip_list = df['zip_code']
    
# Get starting coordinates
    x = df.get_value(df.loc[(df['city'] == USER_CITY) & (df['state'] == USER_STATE_ABV)].index.tolist()[0],'lat')
    y = df.get_value(df.loc[(df['city'] == USER_CITY) & (df['state'] == USER_STATE_ABV)].index.tolist()[0],'long')

# Generate subset of Yelp data for mapping brewery locations
    yelp_sub = yelp_data.loc[(yelp_data['zip_code'].isin(zip_list))]
     
# Choropleth mapping
    geo_str = (GEO_DF[GEO_DF['postal-code'].isin(zip_list)]).to_json()

    threshold_scale = np.linspace(df[METRIC].min(),
                              df[METRIC].max(), 6, dtype=float).tolist()

    mapa = folium.Map(location=[x, y],
                  tiles="cartodbpositron",
                  zoom_start=10)

    yelp_group = folium.FeatureGroup(name='Breweries')

    mapa.choropleth(geo_str=geo_str,
              data=df,
              columns=['zip_code', METRIC],
              fill_color='YlGnBu',
              key_on='feature.properties.postal-code',
              threshold_scale=threshold_scale,
                legend_name=METRIC)

# Add circle and popup info for each city
    for row in yelp_sub.iterrows():
    # Only plot point if lat/long is available.
        if (not np.isnan(row[1]['latitude']) and not np.isnan(row[1]['longitude'])):
            brewery_metadata = """
                <ul>
                    <li><strong>Name</strong>: {0}</li>
                    <li><strong>Rating</strong>: {1}</li>
                    <li><strong>Review Count</strong>: {2}</li>
                    <li><strong>Price ($)</strong>: {3}</li>
                </ul>""".format(
                str(row[1]['name_norm']), str(row[1]['rating']), str(row[1]['review_count']), str(row[1]['price']))
            iframe = branca.element.IFrame(html=brewery_metadata, width=200, height=100)
            popup = folium.Popup(iframe, max_width=2650)
            folium.CircleMarker(
                location = [row[1]['latitude'], row[1]['longitude']],
                color = 'white', fill_color = 'red', 
                fill_opacity = 1,radius = 3,
                popup=popup).add_to(yelp_group)

    mapa.add_child(yelp_group)
    
    folium.LayerControl().add_to(mapa)

    mapa.save('./Cloropleth Mapping/'+city+'_'+state+'_'+metric+'_map_'+str(year)+'.html')

    return mapa

In [147]:
get_map('Hanover','MD','med_inc',2015)

In [398]:
import fiona; fiona.supported_drivers

{'ARCGEN': 'r',
 'AeronavFAA': 'r',
 'BNA': 'raw',
 'DGN': 'raw',
 'DXF': 'raw',
 'ESRI Shapefile': 'raw',
 'GPKG': 'rw',
 'GPSTrackMaker': 'raw',
 'GPX': 'raw',
 'GeoJSON': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'OpenFileGDB': 'r',
 'PCIDSK': 'r',
 'SEGY': 'r',
 'SUA': 'r'}

In [401]:
GEO_DF.to_file('geo_data_all.geojson')